In [18]:
%pylab inline
import dsift
import sys
sys.path.insert(0, "/home/ubuntu/pictureweb/notebooks/")
from pictureweb.distributed import sharded_matrix 
from pictureweb.distributed import distributed as D 
from importlib import reload
reload(sharded_matrix)
import pywren
import scipy.linalg
from pictureweb.opt import ls
from numba import jit, float64, int64
from sklearn import metrics
import pywren

Populating the interactive namespace from numpy and matplotlib


In [140]:
%time X_train_sharded_big_shards = sharded_matrix.ShardedMatrix("X_train_imagenet_fc7", bucket="vaishaalpywrenlinalg")
%time X_train_sharded = sharded_matrix.ShardedMatrix("X_train_imagenet_fc7_small_shards", bucket="vaishaalpywrenlinalg")
%time X_test_sharded = sharded_matrix.ShardedMatrix("X_test_imagenet_fc7", bucket="vaishaalpywrenlinalg")

CPU times: user 64 ms, sys: 4 ms, total: 68 ms
Wall time: 213 ms
CPU times: user 64 ms, sys: 0 ns, total: 64 ms
Wall time: 120 ms
CPU times: user 64 ms, sys: 0 ns, total: 64 ms
Wall time: 131 ms


In [141]:
%time y_train_sharded = sharded_matrix.ShardedMatrix("y_train_imagenet_fc7_small_shards", bucket="vaishaalpywrenlinalg")

CPU times: user 56 ms, sys: 0 ns, total: 56 ms
Wall time: 202 ms


In [4]:
y_train = y_train[:,  np.newaxis]

NameError: name 'y_train' is not defined

In [5]:
y_train

NameError: name 'y_train' is not defined

In [6]:
%time X_train = D.get_local_matrix(X_train_sharded_big_shards)
%time X_test = D.get_local_matrix(X_test_sharded)

CPU times: user 24 ms, sys: 4.6 s, total: 4.62 s
Wall time: 41.6 s
CPU times: user 24 ms, sys: 312 ms, total: 336 ms
Wall time: 3.92 s


In [7]:
y_train = np.load("y_train_fc7.npy")
y_test = np.load("y_val_fc7.npy")

In [8]:
%time XTX = X_train.T.dot(X_train)

CPU times: user 31min 4s, sys: 5.7 s, total: 31min 10s
Wall time: 1min 22s


In [9]:
%time y_train_enc = np.eye(1000)[y_train]

CPU times: user 836 ms, sys: 2.54 s, total: 3.37 s
Wall time: 3.37 s


In [10]:
%time Xty = X_train.T.dot(y_train_enc)

CPU times: user 10min 22s, sys: 2min 45s, total: 13min 7s
Wall time: 25.9 s


In [171]:
%time model = scipy.linalg.solve(XTX, Xty, sym_pos=True)

CPU times: user 8.1 s, sys: 228 ms, total: 8.32 s
Wall time: 1.25 s


In [12]:
%time model2 = np.linalg.inv(XTX).dot(Xty)

CPU times: user 47.1 s, sys: 6.38 s, total: 53.4 s
Wall time: 3.65 s


In [13]:
%time y_train_pred = X_train.dot(model)
%time y_test_pred = X_test.dot(model)

CPU times: user 6min 36s, sys: 3min 5s, total: 9min 41s
Wall time: 21 s
CPU times: user 20.4 s, sys: 13.2 s, total: 33.6 s
Wall time: 1.05 s


In [14]:
%time XTX_inv = np.linalg.inv(XTX)

CPU times: user 43 s, sys: 2 s, total: 45 s
Wall time: 3.86 s


In [15]:
print("Test top 5", ls.top_k_accuracy(y_test, y_test_pred))
print("Train top 5", ls.top_k_accuracy(y_train, y_train_pred))
print("Test top 1", ls.top_k_accuracy(y_test, y_test_pred, k=1))
print("Train top 1", ls.top_k_accuracy(y_train, y_train_pred, k=1))

Test top 5 0.73462
Train top 5 0.845536140097
Test top 1 0.49606
Train top 1 0.612692958841


In [16]:
help(jit)

Help on function jit in module numba.decorators:

jit(signature_or_function=None, locals={}, target='cpu', cache=False, **options)
    This decorator is used to compile a Python function into native code.
    
    Args
    -----
    signature:
        The (optional) signature or list of signatures to be compiled.
        If not passed, required signatures will be compiled when the
        decorated function is called, depending on the argument values.
        As a convenience, you can directly pass the function to be compiled
        instead.
    
    locals: dict
        Mapping of local variable names to Numba types. Used to override the
        types deduced by Numba's type inference engine.
    
    target: str
        Specifies the target platform to compile for. Valid targets are cpu,
        gpu, npyufunc, and cuda. Defaults to cpu.
    
    targetoptions:
        For a cpu target, valid options are:
            nopython: bool
                Set to True to disable the use of Py

In [124]:
def loo_model(XTX_inv, XtY, X_train, y_train, i, n_classes=1000):
    a = X_train[i]
    y = np.eye(n_classes)[y_train[i]]
    Px = np.dot(XTX_inv,a)[:, np.newaxis]
    aPx = np.dot(a.T,Px)
    PxPxt = np.dot(Px,Px.T)
    update = ((1.0/(1.0 - aPx)) * PxPxt)
    XTX_inv_update = XTX_inv + update
    XtY_update = XtY - a[:,np.newaxis].dot(y[np.newaxis,:])
    return np.dot(XTX_inv_update,XtY_update)

In [125]:
def loo_model_eval(XTX_inv, XtY, X_train, y_train, eval_set, n_classes=1000):
    y_preds = np.zeros((len(eval_set), n_classes))
    for idx,i in enumerate(eval_set):
        model_loo = loo_model(XTX_inv, XtY, X_train, y_train.ravel(), i, n_classes)
        y_preds[idx] = X_train[i:i+1].dot(model_loo)
    return y_preds

def loo_model_eval_pywren(XTX_inv_sharded, XtY_sharded, X_train_sharded, y_train_sharded, block, n_classes=1000):
    XTX_inv = XTX_inv_sharded.get_block(0,0)
    XTy = XtY_sharded.get_block(0,0)
    X_train_block = X_train_sharded.get_block(block, 0)
    y_train_block = y_train_sharded.get_block(block, 0)
    return loo_model_eval(XTX_inv, XTy, X_train_block, y_train_block, list(range(X_train_block.shape[0])))
    
        

In [126]:
pwex = pywren.default_executor()

In [139]:
XTX_inv_sharded = sharded_matrix.ShardedMatrix("XTX_inv_sharded", bucket="vaishaalpywrenlinalg", data=XTX_inv, reshard=True)
XTy_sharded = sharded_matrix.ShardedMatrix("XTy_sharded", bucket="vaishaalpywrenlinalg", data=Xty, reshard=True)

Sharding matrix..... of shape (4096, 4096)
0 4096 0 4096
Sharding matrix..... of shape (4096, 1000)
0 4096 0 1000


In [114]:
len(y_train_sharded._block_idxs(0))

20019

In [157]:
futures = pwex.map(lambda x: loo_model_eval_pywren(XTX_inv_sharded, XTy_sharded, X_train_sharded, y_train_sharded, x), y_train_sharded._block_idxs(0)[:3000], exclude_modules=['site-packages'])

In [158]:
%time pywren.wait(futures)

CPU times: user 5min 39s, sys: 17.9 s, total: 5min 56s
Wall time: 12min 4s


([<pywren.future.ResponseFuture at 0x7faa6d48d6d8>,
  ...],
 [])

In [159]:
futures[1].result()

array([[ 0.00530929, -0.00771338,  0.00293294, ...,  0.01098144,
        -0.00362201,  0.00796177],
       [ 0.01018667, -0.00775844, -0.01247368, ...,  0.00682678,
         0.00348112, -0.00560325],
       [-0.01543862,  0.03598055,  0.00898524, ...,  0.00286656,
         0.00232578,  0.01507424],
       ..., 
       [-0.00242012, -0.0075229 ,  0.00922731, ...,  0.00383395,
         0.00010685,  0.01102333],
       [ 0.00434358,  0.00438399,  0.00537604, ..., -0.00205298,
         0.00594325, -0.00405986],
       [-0.00170956,  0.00821895, -0.00374625, ..., -0.00520083,
        -0.00391445, -0.00716681]])

In [160]:
preds = np.vstack([f.result() for f in futures])

In [163]:
reload(D)

<module 'pictureweb.distributed.distributed' from '/home/ubuntu/pictureweb/pictureweb/distributed/distributed.py'>

In [154]:
help(D.get_row_block)

Help on function get_row_block in module pictureweb.distributed.distributed:

get_row_block(X_sharded, row_blocks, dtype='float64', mmap_loc=None)



In [167]:
%time y_train_subset = D.fast_kernel_column_blocks_get(y_train_sharded, mmap_loc="/dev/shm/y_train_subset", col_blocks=[0], row_blocks=y_train_sharded._block_idxs(0)[:3000], workers=100)

CPU times: user 320 ms, sys: 7.4 s, total: 7.72 s
Wall time: 17.7 s


In [170]:
ls.top_k_accuracy(y_train_subset, preds, k=1)

memmap(0.5988854166666666)

In [413]:
y_train_enc = np.eye(1000)[y_train]

In [47]:
y_train.shape

(1281167,)

In [80]:
%time model_loo = loo_model(XTX_inv, Xty, X_train, y_train, 0)
                                 

CPU times: user 8.32 s, sys: 852 ms, total: 9.17 s
Wall time: 290 ms


In [447]:
idxs = [i for i in range(1000)]

In [ ]:
%time y_preds =  loo_model(XTX_inv, Xty, X_train, y_train)

In [457]:
metrics.accuracy_score(np.argmax(y_preds, axis=1), y_train[idxs])

0.58699999999999997

In [454]:
ls.top_k_accuracy(y_train[idxs], y_preds)

0.81999999999999995

In [320]:
np.linalg.norm(np.eye(XTX.shape[0]) - P.dot(XTX + X_train[0].dot(X_train[0]).T))

2.192810176508985e-12

In [ ]:
XTX_inv = np.linalg.inv(XTX)

In [173]:
%time P2 = XTX_inv - ((1.0/(1 + X_train[0].T.dot(XTX_inv).dot(X_train[0]))) * (XTX_inv.dot(X_train[0]).dot(XTX_inv.dot(X_train[0]).T)))

CPU times: user 2.05 s, sys: 148 ms, total: 2.2 s
Wall time: 69.1 ms


In [324]:
np.linalg.norm(np.eye(XTX.shape[0]) - P2.dot(XTX - X_train[0].dot(X_train[0]).T))

17412.026976999598

In [268]:
np.argmax(y_train_pred_loo[3])

637

In [409]:
y_train_pred_loo = X_train.dot(model_loo)
np.argmax(y_train_pred_loo, axis=1)

array([544, 569, 267, ..., 579,  39, 268])

In [410]:
metrics.accuracy_score(np.argmax(y_train_pred_loo, axis=1), y_train)

0.61269295884143127

In [152]:
metrics.accuracy_score(np.argmax(y_train_pred_loo, axis=1), y_train)

CPU times: user 23.3 s, sys: 6.53 s, total: 29.8 s
Wall time: 1.07 s


In [253]:
X_train

memmap([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  3.77013946,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 2.71604156,  0.        ,  0.        , ...,  0.        ,
         0.05119753,  0.        ],
       ..., 
       [ 0.33024526,  0.        ,  0.        , ...,  0.        ,
         0.        ,  1.94778013],
       [ 0.        ,  0.87086219,  3.1447084 , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  2.61814594]])

In [ ]:
np.lin

In [360]:
XTX_inv = np.linalg.inv(XTX)

In [361]:
x = X_train[0]

In [362]:
xxt = x.dot(x.T)

In [363]:
XTX_plus_one = XTX + xxt

In [364]:
XTX_plus_one_inv = np.linalg.inv(XTX_plus_one)

In [365]:
XTX_plus_one_inv

memmap([[  2.08450128e-06,   1.06517459e-08,  -1.70364250e-08, ...,
         -6.00817565e-09,   7.91829213e-09,   3.01785942e-09],
       [  1.06517459e-08,   1.99430265e-06,   1.25169046e-08, ...,
          1.37084192e-08,   1.57383629e-09,  -2.57118505e-09],
       [ -1.70364250e-08,   1.25169046e-08,   1.97613968e-06, ...,
         -5.70736633e-09,   1.72675027e-08,  -8.06020142e-09],
       ..., 
       [ -6.00817565e-09,   1.37084192e-08,  -5.70736633e-09, ...,
          1.85052779e-06,   5.48731163e-10,  -7.45805817e-09],
       [  7.91829213e-09,   1.57383629e-09,   1.72675027e-08, ...,
          5.48731163e-10,   1.79526251e-06,   1.65971726e-08],
       [  3.01785942e-09,  -2.57118505e-09,  -8.06020142e-09, ...,
         -7.45805817e-09,   1.65971726e-08,   1.79838436e-06]])

In [390]:
Px = XTX_inv.dot(x)[:, np.newaxis]

In [391]:
PxPxt = Px.dot(Px.T)

In [392]:
update = (1.0/(1.0 + x.T.dot(Px))*(PxPxt))

In [397]:
(XTX_inv - update)

memmap([[  2.08450726e-06,   1.06575921e-08,  -1.70472893e-08, ...,
         -6.01911768e-09,   7.91348584e-09,   2.98779533e-09],
       [  1.06575921e-08,   1.99430621e-06,   1.25102330e-08, ...,
          1.37004178e-08,   1.57053308e-09,  -2.59250006e-09],
       [ -1.70472893e-08,   1.25102330e-08,   1.97615216e-06, ...,
         -5.69243900e-09,   1.72736713e-08,  -8.02041683e-09],
       ..., 
       [ -6.01911768e-09,   1.37004178e-08,  -5.69243900e-09, ...,
          1.85054443e-06,   5.55775975e-10,  -7.41317311e-09],
       [  7.91348584e-09,   1.57053308e-09,   1.72736713e-08, ...,
          5.55775975e-10,   1.79526547e-06,   1.66160943e-08],
       [  2.98779533e-09,  -2.59250006e-09,  -8.02041683e-09, ...,
         -7.41317311e-09,   1.66160943e-08,   1.79850517e-06]])

In [ ]:
Px.shape